In [5]:
#Imports used throughout the notebook
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

import seaborn as sns
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Moving computations to the gpu if available
device = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else torch.device("mps")
    if torch.backends.mps.is_available()
    else torch.device("cpu")
)
print(f"Using {device} device")

Using cuda device


# Problem 1:
### a: NLP
Please discuss the recent trend of rapidly increasing sizes of NLP architectures. 

### b: Transfer Learning
Classify the following example of transfer learning. More exactly, what are the domains and tasks, and which are being changed?

Source: Using a step counter to monitor exercise in healthy people.

Target: Using a step counter to indicate recovery progression in a patient



### c: Attention

Assume dotproduct attention, and that the hidden states of the encoder layer are [0,1,4],[-1,1,2],[1,1,1],[2,1,1]. If the activation for the previous decoder is [0.1,1,-2], what is the attention-context vector?



### d: Transformers

Explain the 'positional encoding' step for transformers. Why is it done, how is it done?



### e: Bounding box detection:
Given a dataset with two classes; cats and dogs, and the following detections:

TP = True positive
FP = False positive

cat_det = [TP, FP, TP, FP, TP]
pred_scores_cat = [0.7, 0.3, 0.5, 0.6, 0.55]

dog_det = [FP, TP, TP, FP, TP, TP]
pred_scores_dog = [0.4, 0.35, 0.95, 0.5, 0.6, 0.7]

There are in total 3 cats and 4 dogs in the images.

Calculate the mean average precision (mAP)



### f: Semantic segmentation - FCN 1:
Given an image sized 1024x768x3 (width x height x channels), with 7 classes. What is the size of the target image if targets are one-hot encoded?



### g: Semantic segmentation - FCN 2: 
What is a fully-convolutional network? When is it useful?



### h:Residual Networks:
Explain residual layers and their advantage. 



### i: Intersection-Over-Union
Calculate the intersection over union in for these four bounding-boxes and target bounding boxes:

![](bounding-boxes.PNG)



### j: Variational autoencoders:
What are the strengths of a variational autoencoder (VAE) compared to an autoencoder (AE)?

# Problem 2:
Below is attached a script for generating a data set to learn translation of dates between multiple human readable and a machine readable format (ISO 8601). 

Task: Using an encoder-decoder setup, perform translation from human readable to machine readable formats. Please express the performance of your trained network in terms of average accuracy of the translated output (so, accuracy on a per-character basis). 

Restriction: we specifically demand that your presented solution does not include an attention layer. 

 Despite this restriction, the task can be solved in numerous different ways. Here are some examples of solutions of similar problems, for inspiration: 

https://jscriptcoder.github.io/date-translator/Machine%20Translation%20with%20Attention%20model.html

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

Script for generating data set below:

In [3]:
#Copied from dateTrans_student1.py file from brightspace
"""
Created on Mon Oct 18 17:47:38 2021

@author: au207178
"""

#https://www.kaggle.com/eswarchandt/neural-machine-translation-with-attention-dates

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date

from faker import Faker
fake = Faker()

Faker.seed(101)
random.seed(101)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



#%% pytorch dataset

class datesDataset(torch.utils.data.Dataset):
    def __init__(self,locale='da',inputLength=40,outputLength=12, dataSetSize=100):
        
        self.inputLength=inputLength
        self.outputLength=outputLength
        self.length=dataSetSize
        self.lan=locale
        
        self.FORMATS= ['short', # d/M/YY
           'medium', # MMM d, YYY
           'long', # MMMM dd, YYY
           'full', # EEEE, MMM dd, YYY
           'd MMM YYY', 
           'd MMMM YYY',
           'dd/MM/YYY',
           'EE d, MMM YYY',
           'EEEE d, MMMM YYY']
        

        #generate vocabularies:
        alphabet=sorted(tuple('abcdefghijklmnopqrstuvwxyzæøå'))
        numbers=sorted(tuple('0123456789'))
        symbols=['<SOS>','<EOS>',' ',',','.','/','-','<unk>', '<pad>'];
        self.humanVocab=dict(zip(symbols+numbers+alphabet,
                            list(range(len(symbols)+len(numbers)+len(alphabet)))))
        self.machineVocab =dict(zip(symbols+numbers,list(range(len(symbols)+len(numbers)))))
        self.invMachineVocab= {v: k for k, v in self.machineVocab.items()}

    def string_to_int(self,string, length, vocab):
        string = string.lower()
        

        if not len(string)+2<=length: #+2 to make room for SOS and EOS
            print(len(string),string)
            print('Length:',length)
            
            raise AssertionError()

        
        rep = list(map(lambda x: vocab.get(x, '<unk>'),string))
        rep.insert(0,vocab['<SOS>']); rep.append(vocab['<EOS>']) #add start and of sequence
        
        if len(string) < length:
            rep += [vocab['<pad>']] * (length - len(rep))
        
        return rep        
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, idx):
        dt = fake.date_object()

        date = format_date(dt, format=random.choice(self.FORMATS), locale=self.lan)
        human_readable = date.lower().replace(',', '')
        machine_readable = dt.isoformat()
        
        humanEncoded=torch.LongTensor(self.string_to_int(human_readable,self.inputLength,self.humanVocab))
        machineEncoded=torch.LongTensor(self.string_to_int(machine_readable,self.outputLength,self.machineVocab))
        

        
        return human_readable, machine_readable, humanEncoded,machineEncoded

e=datesDataset()
human_readable, machine_readable, humanEncoded,machineEncoded=e[0]